In [7]:
import gymnasium as gym
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy
import optuna

In [8]:
def make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight):
    """
    Crea e restituisce l'ambiente HalfCheetah-v5 dalla libreria Gymnasium con i parametri specificati.
    """
    return gym.make("HalfCheetah-v5", 
                    reset_noise_scale=reset_noise_scale, 
                    forward_reward_weight=forward_reward_weight, 
                    ctrl_cost_weight=ctrl_cost_weight)

# Hyperparameter tuning con Optuna per SAC
def objective(trial):
    # Parametri dell'environment
    reset_noise_scale = trial.suggest_float('reset_noise_scale', 0.005, 0.3)  
    forward_reward_weight = trial.suggest_float('forward_reward_weight', 0.7, 1.5)
    ctrl_cost_weight = trial.suggest_float('ctrl_cost_weight', 0.02, 0.3)

    # Crea l'ambiente
    env = make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

    # Iperparametri per il modello SAC
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
    buffer_size = trial.suggest_categorical('buffer_size', [100000, 300000, 500000])
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256, 512])
    tau = trial.suggest_float('tau', 0.005, 0.02)
    gamma = trial.suggest_float('gamma', 0.95, 0.9999)

    # Per l’entropia, possiamo scegliere tra un coefficiente fisso oppure "auto"
    # Se si desidera variare su diversi valori, possiamo usare una scelta categoriale
    ent_coef = trial.suggest_categorical('ent_coef', ['auto', 0.001, 0.01, 0.05])

    # Frequenza di training e gradient steps
    train_freq = trial.suggest_categorical('train_freq', [1, 64, 256, 512])
    gradient_steps = trial.suggest_categorical('gradient_steps', [1, 8, 16, 32])

    # Crea ed allena il modello SAC
    model = SAC(
        "MlpPolicy",
        env,
        learning_rate=learning_rate,
        buffer_size=buffer_size,
        batch_size=batch_size,
        tau=tau,
        gamma=gamma,
        train_freq=train_freq,          # Quante azioni eseguite prima di un update
        gradient_steps=gradient_steps,  # Numero di passate di gradienti dopo ogni train_freq
        ent_coef=ent_coef,
        verbose=0,
    )

    model.learn(total_timesteps=30_000)

    # Disabilita gli aggiornamenti di normalizzazione per la valutazione
    env.training = False
    env.norm_reward = False

    # Valuta il modello su 100 episodi in modo deterministico
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=60, deterministic=True)

    print(f'Mean reward: {mean_reward}')
    return mean_reward

# Crea lo studio Optuna e ottimizza l'obiettivo
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)

# Stampa i migliori iperparametri trovati
print("Best hyperparameters:", study.best_params)


[I 2025-02-23 12:09:32,075] A new study created in memory with name: no-name-4fedb24e-23b0-4a7e-a95a-11a19de9b6b5
/var/folders/d5/8f8lxk3x1gddrb19zv8xl57m0000gn/T/ipykernel_61413/2280089131.py:23: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
[I 2025-02-23 12:09:50,441] Trial 0 finished with value: -1.955694197690328 and parameters: {'reset_noise_scale': 0.13878910117362878, 'forward_reward_weight': 1.4650717081534432, 'ctrl_cost_weight': 0.2885979897914906, 'learning_rate': 8.629085657598991e-06, 'buffer_size': 500000, 'batch_size': 128, 'tau': 0.007169229553311472, 'gamma': 0.9507766264673688, 'ent_coef': 0.05, 'train_freq': 256, 'gradient_steps': 1}. Best is trial 0 with value: -1.955694197690328.


Mean reward: -1.955694197690328


[I 2025-02-23 12:10:16,637] Trial 1 finished with value: -49.78990560391394 and parameters: {'reset_noise_scale': 0.08443024244425591, 'forward_reward_weight': 1.1797300236439838, 'ctrl_cost_weight': 0.14916472656689986, 'learning_rate': 8.9542020972875e-06, 'buffer_size': 100000, 'batch_size': 256, 'tau': 0.009848926871440445, 'gamma': 0.9620054855796288, 'ent_coef': 0.01, 'train_freq': 512, 'gradient_steps': 32}. Best is trial 0 with value: -1.955694197690328.


Mean reward: -49.78990560391394


[I 2025-02-23 12:10:39,868] Trial 2 finished with value: -1.4428011602217927 and parameters: {'reset_noise_scale': 0.20069101158791108, 'forward_reward_weight': 1.472253859983883, 'ctrl_cost_weight': 0.03868385263270402, 'learning_rate': 9.18405793543639e-06, 'buffer_size': 100000, 'batch_size': 512, 'tau': 0.019060006655289487, 'gamma': 0.9593327700351577, 'ent_coef': 'auto', 'train_freq': 256, 'gradient_steps': 8}. Best is trial 2 with value: -1.4428011602217927.


...

Mean reward: 3218.4274166426017


[I 2025-02-23 18:23:40,644] Trial 23 finished with value: 2682.989667459523 and parameters: {'reset_noise_scale': 0.169196947073717, 'forward_reward_weight': 1.0277916449907698, 'ctrl_cost_weight': 0.24714451661754258, 'learning_rate': 5.522215108386716e-05, 'buffer_size': 300000, 'batch_size': 128, 'tau': 0.012274620888838465, 'gamma': 0.9915516919961503, 'ent_coef': 0.001, 'train_freq': 1, 'gradient_steps': 8}. Best is trial 14 with value: 3254.8171763902315.


Mean reward: 2682.989667459523


[W 2025-02-23 18:29:06,380] Trial 24 failed with parameters: {'reset_noise_scale': 0.16731841286058588, 'forward_reward_weight': 1.0594943649394934, 'ctrl_cost_weight': 0.23657468751702668, 'learning_rate': 0.0002779745914996516, 'buffer_size': 300000, 'batch_size': 128, 'tau': 0.011876732693483865, 'gamma': 0.9905317825475765, 'ent_coef': 0.001, 'train_freq': 1, 'gradient_steps': 8} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/d5/8f8lxk3x1gddrb19zv8xl57m0000gn/T/ipykernel_61413/2280089131.py", line 52, in objective
    model.learn(total_timesteps=30_000)
  File "/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/

KeyboardInterrupt: 